In [1]:
'''
Applicant: Matthew Kwok
'''

'\nApplicant: Matthew Kwok\n'

In [2]:
import pandas as pd

In [3]:
# import csv file
file_name = '2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv'

df = pd.read_csv(file_name)

In [4]:
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [5]:
# convert created_at to just date
df['date'] = pd.to_datetime(df['created_at'], format = '%Y-%m-%d').dt.date

In [6]:
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,date
0,1,53,746,224,2,cash,2017-03-13 12:36:56,2017-03-13
1,2,92,925,90,1,cash,2017-03-03 17:38:52,2017-03-03
2,3,44,861,144,1,cash,2017-03-14 4:23:56,2017-03-14
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,2017-03-26
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,2017-03-01


In [7]:
# check for NaN in columns
df.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
date              0
dtype: int64

In [8]:
# number of unique shops
df.shop_id.nunique()

100

In [9]:
# total number of entries/rows
len(df)

5000

In [10]:
# df['order_amount'].sum() / len(df['order_amount'])
# 15725640 / 5000

'''
Value is highly skewed by shops who may have had much
more sales than other stores.
'''

df['order_amount'].mean()

3145.128

In [11]:
'''
Since each day there are different number of orders with different order amounts at different stores,
it may be beneficial to look at the AOVs by shops since some stores may have more sales than others
'''
df1 = df[['date', 'shop_id','order_id', 'order_amount', 'total_items']].sort_values('date')
df1

,date,shop_id,order_id,order_amount,total_items
1143,2017-03-01,3,1144,296,2
1610,2017-03-01,25,1611,260,2
1608,2017-03-01,18,1609,156,1
3146,2017-03-01,28,3147,492,3
3152,2017-03-01,46,3153,166,1
...,...,...,...,...,...
842,2017-03-30,39,843,402,3
3988,2017-03-30,27,3989,169,1
882,2017-03-30,23,883,624,4
954,2017-03-30,37,955,568,4


In [12]:
'''
want to see the total $ amount and total volume for each store over the 30 days (March 2017)
For example shop 1 had a revenue of $13588, selling 86 shoes
'''
# group by shop and sum across the columns
# reset index to remove multiindex
df1_sum = df1.groupby('shop_id').sum().reset_index().drop(['order_id'], axis = 1)
df1_sum

,shop_id,order_amount,total_items
0,1,13588,86
1,2,9588,102
2,3,14652,99
3,4,13184,103
4,5,13064,92
...,...,...,...
95,96,16830,110
96,97,15552,96
97,98,14231,107
98,99,18330,94


In [13]:
'''
divide by 30 and not use the mean() because the mean()
divides by the number of observations/orders in the numerator
'''
# calculates the aov per day
period = 30   # daily in a month

data = {
    df1_sum.columns[0]: df1_sum['shop_id'],                                # column 1
    'Average Order Value ($) per day': df1_sum['order_amount'] / period,   # column 2
    'Average Order Volume per day': df1_sum['total_items'] / period        # column 3
}

df_aov = pd.DataFrame(data)
df_aov

,shop_id,Average Order Value ($) per day,Average Order Volume per day
0,1,452.933333,2.866667
1,2,319.600000,3.400000
2,3,488.400000,3.300000
3,4,439.466667,3.433333
4,5,435.466667,3.066667
...,...,...,...
95,96,561.000000,3.666667
96,97,518.400000,3.200000
97,98,474.366667,3.566667
98,99,611.000000,3.133333


In [14]:
'''
Answer to b. and c. I would look at the volume metric, in other words
on average how many shoes did each store sell per day. 


Can see that shop 42 had the highest order $ amount per day in the month of March.
Shop 92 had the least order amount ($).
Interestingly, shop 78 had the second highest $ amount per day but the volume of shoes
sold is much less. This could imply that shop 78 is selling much pricer shoes. 
'''

# sorts the aov values in descending order
df_aov.sort_values(by = df_aov.columns[1], ascending = False)

,shop_id,Average Order Value ($) per day,Average Order Volume per day
41,42,399672.533333,1135.433333
77,78,75460.000000,2.933333
88,89,770.933333,3.933333
80,81,755.200000,4.266667
5,6,754.233333,4.033333
...,...,...,...
1,2,319.600000,3.400000
99,100,284.900000,2.566667
55,56,269.100000,2.300000
31,32,265.966667,2.633333
